In [ ]:
#import streamlit as st
import gradio as gr
import requests
from langchain_core.tools import tool
import os
import requests
import json
from typing import TypedDict, Annotated, List
from langgraph.graph import StateGraph, END
from langchain_core.messages import HumanMessage, AIMessage
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from langgraph.prebuilt import create_react_agent
from pprint import pformat
from langchain.tools import tool
from serpapi import GoogleSearch
from langchain_community.utilities import ArxivAPIWrapper,WikipediaAPIWrapper
from langchain_community.tools import ArxivQueryRun,WikipediaQueryRun
from pprint import pformat

class PlannerState(TypedDict):
  messages : Annotated[List[HumanMessage | AIMessage], "the messages in the conversation"]
  city: str
  interests: List[str]
  itinerary: str

def search_internet(query: str) -> str:
  "Search the google api for best results "
  params = {
  "q": query,
  "api_key": "",
  "num":2
}
  search = GoogleSearch(params)
  results = search.get_dict()
  organic_results = results["organic_results"]
  formatted_results = ""
  for result in organic_results:
      formatted_results += f"Title: {result['title']}\n"
      formatted_results += f"Link: {result['link']}\n"
      formatted_results += f"Snippet: {result['snippet']}\n\n"

  return formatted_results

llm=ChatGroq(groq_api_key='',model_name="llama-3.3-70b-versatile")

def format_itinerary(itinerary_list):
    """
    Formats a list of itinerary items into a human-readable string.

    Args:
        itinerary_list: A list of strings representing the itinerary.

    Returns:
        A formatted string representing the itinerary.
    """

    formatted_itinerary = ""
    current_day = ""

    for item in itinerary_list:
        # Check for day headers
        if item.startswith("Day"):
            current_day = item + "\n"
            formatted_itinerary += current_day
        # Check for empty lines and skip them
        elif item == "":
            continue
        # Format itinerary items with time and activity
        else:
            formatted_itinerary += "- " + item + "\n"

    return formatted_itinerary

def chat_with_agent(city: str,interests:str):

  state = {
        "messages": [],
        "city": city,
        "interests": interests,
        "itinerary": "",
    }

  arxiv_wrapper=ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=300)
  arxiv_tool=ArxivQueryRun(api_wrapper=arxiv_wrapper)

  api_wrapper=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=300)
  wiki_tool=WikipediaQueryRun(api_wrapper=api_wrapper)

  tools=[wiki_tool,search_internet]
  graph = create_react_agent(llm, tools=tools)

  initial_message = HumanMessage(content=f"You are a helpful travel planner.Plan a 3 day trip based on weather conditions or if user suggests to {city} based on these interests: {interests}.Provide a detailed elaboartive itinerary with time slots.")

  # # Update the state with the initial message
  state["messages"].append(initial_message)

  events = graph.stream(state, stream_mode="values")

  for event in events:
    response_text= event["messages"][-1].content

  formatted_response = format_itinerary(response_text.split('\n'))

  return formatted_response if response_text else "⚠️ No response received."


def create_interface():
    with gr.Blocks(css="body {background-color: #f7f7f7;}") as iface:
        gr.Markdown("<h1 style='text-align: center;'> 🌍✨ Travel Planner </h1>")

        with gr.Row():
            with gr.Column(scale=1, min_width=250):
                gr.Image("https://cdn-icons-png.flaticon.com/512/201/201623.png", width=100, show_label=False)

            with gr.Column(scale=3):
                gr.Markdown(
                    "<p style='text-align: center; font-size: 18px; color: #444;'>"
                    "Plan your perfect trip with personalized recommendations! 🚀"
                    "</p>"
                )

        with gr.Row():
            with gr.Column(scale=1, min_width=300):
                city_input = gr.Textbox(label="🌆 Enter the city:", placeholder="e.g., Paris, Tokyo, New York", interactive=True)
                interests_input = gr.Textbox(label="🎭 Enter your interests:", placeholder="e.g., Food, Art, Adventure", interactive=True)

            with gr.Column(scale=1, min_width=100, elem_id="button-column"):
                submit_button = gr.Button("🚀 Plan My Trip!", size="lg", variant="primary")

        output = gr.Textbox(label="🗺️ Your Travel Plan:", interactive=False, lines=3)

        submit_button.click(chat_with_agent, inputs=[city_input, interests_input], outputs=output)

    return iface


# Launch the interface
iface = create_interface()
iface.launch(debug=True)
#search_internet("Famous places in kota?")
#chat_with_agent("kota","kachori,7 wonders")

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://810f19751b0a2a29de.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://810f19751b0a2a29de.gradio.live


In [ ]:
#!pip install langgraph langsmith langchain langchain_groq langchain_community langgraph-prebuilt gradio
#!pip install google-search-results
#!pip install arxiv wikipedia
!pip install pprint

ERROR: Could not find a version that satisfies the requirement pprint (from versions: none)
ERROR: No matching distribution found for pprint


In [ ]:
# def create_interface():
#     with gr.Blocks() as iface:
#         gr.Markdown("# 🤖 Travel Planner ")

#         with gr.Row():
#             city_input = gr.Textbox(label="Enter the city:")
#             interests_input = gr.Textbox(label="Enter your interests:")

#         with gr.Row():
#             submit_button = gr.Button("Send")

#         output = gr.Textbox(label="Response:")

#         submit_button.click(chat_with_agent, inputs=[city_input,interests_input], outputs=output)

#     return iface

NameError: name 'rary' is not defined